# Deep Learning and Inverse Problems
### Summer 2025
### Reinhard Heckel
## **Chapter 6 exercise**


## Denoising Images with a U-Net

In this exercise, your task is to train a U-Net for image denoising using PyTorch. As dataset, we use the Berkeley Segmentation Dataset (BSDS300). This dataset contains 300 clean color images. For simplicity, we consider Gaussian noise and convert the images to grayscale (see below). The attached file `unet.py` contains a PyTorch implemenation of the U-Net for you to use.

### Download Dataset
Running these commands in the terminal downloads the BSDS300 dataset as a `.tgz` file, unzip it and delete the `.tgz` file.

In [10]:
#!wget https://www2.eecs.berkeley.edu/Research/Projects/CS/vision/bsds/BSDS300-images.tgz
#!tar -xvzf BSDS300-images.tgz
#!rm BSDS300-images.tgz

### Optional: Google Colab
If you want to use google colab for free GPU use, you can copy the notebook, unet.py and the dataset to your google drive (here in directory 'Colab Notebooks'), and mount the drive like this.

In [11]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
cd /content/drive/MyDrive/'Colab Notebooks'

### Import packages
We want you to use PyTorch in this exercise, as it is the most common library for deep learning.

In [ ]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
import tqdm  # for nice progress bars
from matplotlib import pyplot as plt
# TODO: maybe you want to use additional libraries

from unet import Unet

### Build PyTorch Datasets and Dataloaders

We use all 200 images in `./BSDS300/images/train` for training. The first 50 images in `./BSDS300/images/test` are used for validation and the remaining 50 for testing.

In [ ]:
dataset_dir = "./BSDS300"

train_set_dir = f"{dataset_dir}/images/train"
train_img_files = [f"{train_set_dir}/{filename}" for filename in os.listdir(train_set_dir)]
# use this to train with fewer data
# train_img_files = random.sample(train_img_files, 50)

test_set_dir = f"{dataset_dir}/images/test"
test_img_files = [f"{test_set_dir}/{filename}" for filename in os.listdir(test_set_dir)]
val_img_files = test_img_files[:50]
test_img_files = test_img_files[50:]

### Problem 1
We first implement a `torch.utils.data.Dataset` that returns pairs of noisy images and clean ground truth. 

Complete the code below so that the dataset returns images that are **grayscale** (BSDS contains RGB images, so you have to convert them) with pixel-range $[0,1]$. Add zero-mean Gaussian noise with variance `noise_var` to the clean images. To reduce computational cost, we also want to be able to use chunks instead of full-sized images for training. Therefore, implement code to split an image into non-overlapping chunks of size `(chunk_size, chunk_size)`. For example: An image of size `(512, 512)` should be split into 16 non-overlapping chunks of size `(128,128)`. If any of the image dimension is not divisible by `chunk_size`, simply crop the dimension until it is divisble by `chunk_size`. For reference: If you split all 200 train images into chunks of size `(128,128)`, you should end up with 1200 chunks.

In [ ]:
class NoisyImageChunkDataset(Dataset):
    def __init__(self, img_files, noise_var, chunk_size):
        self.img_files = img_files
        self.noise_var = noise_var
        self.chunk_size = chunk_size
        self.chunks_clean, self.chunks_noisy = self.get_clean_and_noisy_chunks()

    def get_clean_and_noisy_chunks(self):
        # TODO
        return chunks_clean, chunks_noisy

    def __len__(self):
        return len(self.chunks_clean)

    def __getitem__(self, idx):
        return self.chunks_noisy[idx], self.chunks_clean[idx]

In [ ]:
noise_var = 0.005  # more noise makes denoising harder; we suggest you keep this value but you can also experiment with more or less noise
train_chunk_size = ?  # depends on your hardware; larger chunks require more memory during gradient computation; we recommend 128

train_set = NoisyImageChunkDataset(img_files=train_img_files, noise_var=noise_var, chunk_size=train_chunk_size)
# for validation and testing, we do not have to split the images into chunks because we do not have to compute gradients
# the images have shape (321, 481) or (481, 321) so we crop them to (321, 321) to facilitate data loading
val_set = NoisyImageChunkDataset(img_files=val_img_files, noise_var=noise_var, chunk_size=321)
test_set = NoisyImageChunkDataset(img_files=test_img_files, noise_var=noise_var, chunk_size=321)

plt.imshow(train_set[0][0], cmap="gray")

In [ ]:
batch_size = ?  # depends on your hardware

train_loader = DataLoader(train_set, batch_size=batch_size)
val_loader = DataLoader(val_set, batch_size=batch_size)
test_loader = DataLoader(test_set, batch_size=batch_size)

In [ ]:
# more pooling layers and convolutional kernels increase the complexity of the U-Net (see lecture notes)
num_pool_layers = ?
chans = ?
device = "cpu"  # set to "cuda" or "cuda:0" if you have access to a GPU (e.g. via Google Colab)

model = Unet(
    in_chans=1,  # 1 input channel as we use grayscale images as input
    out_chans=1,  # 1 output channel as the model returns grayscale images
    num_pool_layers=num_pool_layers,
    chans=chans
)
model = model.to(device)

### Problem 2
Choose a suitable loss for training and implement the training loop. Be sure to also check the validation loss (or PSNR) every few epochs to be sure that your model does not overfit.

Hint: In deep learning it is often helpful to normalize the data before passing it through the model. In image-to-image tasks (such as denoising), we often also de-normalize the model output to map the pixels back into the original range. For example:
````
mean, std = mean(img_noisy), std(img_noisy)
img_noisy = (img-mean) / std
img_denoised = model(img_noisy)
img_denoised = img_denoised * std + mean
````
An intuition behind this is that the model can then receive and return images in a fixed range which makes learning easier. Feel free to train your model with and without normalization to see if it makes a difference.


In [ ]:
optimizer = ?  # choose a suitable optimizer form torch.optim; we recommend to use the ADAM optimizer

epochs = ?  # how many epochs to train
check_val_every_epochs = ?


for e in range(epochs):
    for imgs_noisy, imgs_clean in tqdm.tqdm(train_loader, desc="Training"):
        imgs_noisy = imgs_noisy.to(device)
        imgs_clean = imgs_clean.to(device)
        #  TODO
    
    if e % check_val_every_epochs == 0:
        # disable gradient computation for validation
        with torch.no_grad():
            for imgs_noisy, imgs_clean in tqdm.tqdm(val_loader, desc="Validation"):
                # TODO

### Problem 3
Test your model on the `test_loader`. Use suitable image metrics (e.g. PSNR, SSIM) to quantitatively assess the denoising performance of your model.

In [ ]:
# TODO